In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train.head()

In [ ]:
#look at bitcoin
btc = train.loc[train.Asset_ID == 1].set_index('timestamp')

In [ ]:
btc.info(show_counts =True)
btc.isna().sum()
(btc.index[1:]-btc.index[:-1]).value_counts().head()

In [ ]:
#fill gaps in data
btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')
a = (btc.index[1:]-btc.index[:-1]).value_counts().head()

In [ ]:
a.iloc[0]
a

In [ ]:
btc

In [ ]:
nan_btc = btc.loc[btc['Target'].isnull()]
(nan_btc.index[1:]-nan_btc.index[:-1]).value_counts().head()

In [ ]:
nan_btc.Target.unique()

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(nan_btc.index, np.ones(len(nan_btc.index)))

In [ ]:
#compare to coins target nan
coins = train.Asset_ID.unique()
for i in coins:
    coin = train.loc[train.Asset_ID == i].set_index('timestamp')
    coin = coin.reindex(range(coin.index[0],coin.index[-1]+60,60),method='pad')  #fil missing values
    nan_coin = coin.loc[coin['Target'].isna()]
    print('coin = ',i, coin.shape, coin.index[0], coin.index[len(coin.index)-1], '# of NAN = ', nan_coin.shape[0], nan_coin.shape[0]/coin.shape[0]*100,"%")
    fig = plt.figure()
    plt.scatter(nan_coin.index, np.ones(len(nan_coin.index)))
    plt.title(i)
    
# coins don't have similar data set lengths
# minimal data is from coin 4, use these timestamps to define training data (1285047, 9) 1555079640 1632182400

# coin 10 has a lot of nan (64%) hardly possible to find overlapping training for which all coins have a valid target

In [ ]:
btc.loc[btc.index == 1571201820]#[1571201700-60:1571201820+60]

In [ ]:
btc.iloc[1956960-861:1956960]

In [ ]:
def calcR(train, index):
    t16 = train['Close'].iloc[index + 16]
    t1 = train['Close'].iloc[index+1]
    R = np.log(t16/t1)
    print(t16, t1, R)
    return R

(calcR(btc, 0))

In [ ]:
def ResidualizeMarket(df, mktColumn, window):
  if mktColumn not in df.columns:
    return df

  mkt = df[mktColumn]

  num = df.multiply(mkt.values, axis=0).rolling(window).mean().values  #numerator of linear regression coefficient
  denom = mkt.multiply(mkt.values, axis=0).rolling(window).mean().values  #denominator of linear regression coefficient
  beta = np.nan_to_num( num.T / denom, nan=0., posinf=0., neginf=0.)  #if regression fell over, use beta of 0

  resultRet = df - (beta * mkt.values).T  #perform residualization
  resultBeta = 0.*df + beta.T  #shape beta

  return resultRet.drop(columns=[mktColumn]), resultBeta.drop(columns=[mktColumn])